# Model complicated

## Read data

In [1]:
import pandas as pd

In [2]:
police_stations = pd.read_json("./new_data/police_stations.json")
community_areas = pd.read_json("./new_data/community_areas.json")

In [3]:
police_stations.head()

,station_name,community_area,longitude,latitude,ID_PS,D_distances,d_distances
0,3510 S Michigan Ave,DOUGLAS,-87.623395,41.830702,1,"[2092.19, 2103.08, 1560.8, 1750.1, 1439.87, 13...","[0, 989.7, 1360.4, 1809.6, 1227.2, 2021.8, 161..."
1,1160 N Larrabee St,NEAR NORTH SIDE,-87.643352,41.903242,2,"[1217.56, 1283.21, 686.17, 930.23, 662.57, 467...","[989.7, 0, 487.2, 992.2, 1827.8, 1154.4, 1160...."
2,850 W Addison St,LAKE VIEW,-87.651512,41.947400,3,"[784.26, 885.85, 252.87, 540.93, 298.23, 106.0...","[1360.4, 487.2, 0, 615.9, 2216.9, 709.2, 1220...."
3,5400 N Lincoln Ave,LINCOLN SQUARE,-87.692845,41.979550,4,"[478.94, 346.67, 482.46, 132.04, 511.79, 642.4...","[1809.6, 992.2, 615.9, 0, 2637.8, 373.8, 1185...."
4,1900 W Monterey Ave,MORGAN PARK,-87.668520,41.691435,5,"[3025.14, 3036.02, 2493.75, 2683.05, 2372.81, ...","[1227.2, 1827.8, 2216.9, 2637.8, 0, 2951.4, 25..."


In [4]:
community_areas.head()

,ID_CA,community_area,population,area,neighbours,criminality_index
0,1,ROGERS PARK,55628,4.77,"[1, 2, 77]",0.017029
1,2,WEST RIDGE,77122,9.14,"[1, 2, 4, 13, 77]",0.016114
2,3,UPTOWN,57182,6.01,"[3, 4, 5, 6, 77]",0.016228
3,4,LINCOLN SQUARE,40494,6.63,"[2, 3, 4, 5, 6, 13, 14, 16, 77]",0.008787
4,5,NORTH CENTER,35114,5.31,"[3, 4, 5, 6, 7, 14, 16, 21, 22]",0.005632


## Model 3

**CONSTANTS**
<br>
$I$: Number of community areas = 77 <br>
$J$: Number of police districts or police stations = 23 <br>
$K$: Number of police areas = 5 <br><br>

**DECISION VARIABLES**
<br>
$X_{i, j, k}$: 1 if police area _k_ coordinates police station _j_, which patrols community area _i_, 0 otherwise; $X_{i, j, k} \in \{0,1\}$; _i_ = 1..._I_; _j_ = 1..._J_; _k_ = 1..._K_ <br>
$Y_{j, j', k}$: 1 if police station _j_ and _j'_ are coordinated by police area _k_, 0 otherwise; $Y_{j, j', k} \in \{0,1\}$; _j_ = 1..._J_; _j'_ = 1..._J_; _k_ = 1..._K_ <br>

**Derived variables**
<br>
$w_j$: Workload of police station _j_ $\equiv w_j = \sum_{i=1}^{I} \sum_{k=1}^{K} X_{i, j, k} * C_i$; $w_{j} \in \mathbb{R}$; $w_j \geq 0$; _j_ = 1..._J_ <br>
$v_k$: Workload of police area _k_ $\equiv v_k = \sum_{i=1}^{I} \sum_{j=1}^{J} X_{i, j, k} * C_i$; $v_{k} \in \mathbb{R}$; $v_k \geq 0$; _k_ = 1..._K_ <br><br>


**PARAMETERS**
<br>
$C_i$: Crime index in community area _i_; $C_i \in \mathbb{R}$; _i_=1..._I_ <br>
$D_{i,j}$: Distance from police station _j_ to community area centroid _i_;  $D_{i, j} \in \mathbb{R}$; _i_ = 1..._I_; _j_ = 1..._J_ <br>
$d_{j, j'}$: Distance from police station _j_ to police station _j'_; $d_{j, j'} \in \mathbb{R}$; _j_ = 1.._J_; _j'_ = 1.._J_ <br>
$λ$: Parameter that weights the importance of distances within the cluster; $λ \in \mathbb{R}$; $λ \geq 0$ <br>
$T$: Maximum travel time for a police station to reach the centroid of a community area; $T \in \mathbb{R}$; $T \geq 0$ <br>
$t$: Maximum travel time between two police stations within the same police area; $t \in \mathbb{R}$; $t \geq 0$ <br><br>


**OBJECTIVE FUNCTION**: Minimizing the distance from police stations to community areas and between police stations within the same police area
<br>
min Z =  $\sum_{i=1}^{I} \sum_{j=1}^{J} \sum_{k=1}^{K}X_{i,j,k}*D_{i,j}$ + λ $\sum_{j=1}^{J} \sum_{j'=1}^{J} \sum_{k=1}^{K} Y_{j, j', k}* d_{j, j'}$ <br><br>


**S.T.**:
<br>

**_Cardinality constraints_**
<br>
[Community areas] $\sum_{j=1}^{J} \sum_{k=1}^{K} X_{i, j, k} = 1$; _i_ = 1..._I_ ‎ ‎ # Each community area is patrolled by one police station <br> 
[Police stations] $\sum_{i=1}^{I} \sum_{k=1}^{K} X_{i, j, k} \geq 1$; _j_ = 1..._J_ ‎ ‎ # Each police station patrols at least one community area <br>
[Police areas] $\sum_{i=1}^{I} \sum_{j=1}^{J} X_{i, j, k} \geq 1$; _k_ = 1..._K_ ‎ ‎ # Each police area coordinates at least one police station <br>
[Police area-Police station] $\sum_{k=1}^K X_{i, j, k} = 1; i=1..I, j = 1..J$ ‎ ‎ # A police station is only coordinated by one police area <br>

**_Max number of X in charge of constraints_** 
<br>
[Max CA at j]  $\sum_{i=1}^{I}\sum_{k=1}^{K} X_{i, j, k} \leq 4$; _j_ = 1..._J_ ‎ ‎ # Each police station does NOT patrol more than 4 community areas <br>
[Max PS at k]  $\sum_{i=1}^{I}\sum_{j=1}^{J} X_{i, j, k} \leq 5$; _k_ = 1..._K_ ‎ ‎ # Each police area does NOT coordinate more than 5 police stations <br>

**_Distance constraints_**
<br>
[CA i, PS j] $\sum_{k=1}^K X_{i,j, k} * D_{i,j} \leq T$; _i_ = 1..._I_; _j_ = 1..._J_ ‎ ‎ # Each police station only patrols a community area if it is not too far, at _T_ seconds distance maximum <br> 
[PS j, PA k] $Y_{j,j', k} * d_{j,j'} \leq t $; _j_ = 1..._J_; _j'_ = 1..._J = _j_, _k_ = 1.._K_ ‎ ‎ # Police area coordinates police stations that are not too far between them <br>

**_Workload division constraints_**
<br>---z 0.0.15 PARAMETRO
[Job division at j] $ -0.015 \leq w_j - \frac{\sum_{j=1}^{J}w_j}{J} \leq 0.015$; _j_ = 1..._J_ ‎ ‎ # Balanced job division at police stations <br>
[Job division at k] $ -0.015 \leq v_k - \frac{\sum_{k=1}^{K}v_k}{K} \leq 0.015$; _k_ = 1..._K_ ‎ ‎ # Balanced job division at police areas <br>
[Job j] $ \sum_{i \in {5, 41, 47, 49, 65, 75}} \sum_{k=1}^{K} X_{i,j,k} \leq 1$; _j_ = 1..._J_ ‎ ‎ # Each police station can patrol only one community area with a high crime index <br>

**_Linking constraints:_**
<br>
[Linking] $X_{i, j, k} \leq \sum_{j'=1}^J Y_{j, j', k} $ i = 1..I; j..J; k..K; # If $X_{i, j, k} = 1 and X_{i', j', k} = 1, then Y_{j, j', k} should be active too$


## Code

In [5]:
from ortools.linear_solver import pywraplp

### Solver

In [17]:
solver = pywraplp.Solver.CreateSolver("BOP") ## SCIP suele ser más robusto, CP-SAT, CBC (CUALQUIER VARIABLE)

"""
* Si todas las variables son binarias --> BOP
* Con las que NO hacer experimentos es con GLOP ni CLP al ser solver de programación lineal que asumen que todas son continuas
* Podemos probar solver comerciales que suelen ir más rápido

"""

'\n* Si todas las variables son binarias --> BOP\n* Con las que NO hacer experimentos es con GLOP ni CLP al ser solver de programación lineal que asumen que todas son continuas\n* Podemos probar solver comerciales que suelen ir más rápido\n\n'

### Constants
$I$: Number of community areas = 77 <br>
$J$: Number of police districts or police stations = 23 <br>
$K$: Number of police areas = 5

In [18]:
I, J, K = 77, 23, 5

### Decision variables
$X_{i, j, k}$: 1 if police area _k_ coordinates police station _j_, which patrols community area _i_, 0 otherwise; $X_{i, j, k} \in \{0,1\}$; _i_ = 1..._I_; _j_ = 1..._J_; _k_ = 1..._K_ <br>
$Y_{j, j', k}$: 1 if police station _j_ and _j'_ are coordinated by police area _k_, 0 otherwise; $Y_{j, j', k} \in \{0,1\}$; _j_ = 1..._J_; _j'_ = 1..._J_; _k_ = 1..._K_ 
#### Derived variables
$w_j$: Workload of police station _j_ $\equiv w_j = \sum_{i=1}^{I} \sum_{k=1}^{K} X_{i, j, k} * C_i$; $w_{j} \in \mathbb{R}$; $w_j \geq 0$; _j_ = 1..._J_ <br>
$v_k$: Workload of police area _k_ $\equiv v_k = \sum_{i=1}^{I} \sum_{j=1}^{J} X_{i, j, k} * C_i$; $v_{k} \in \mathbb{R}$; $v_k \geq 0$; _k_ = 1..._K_

In [19]:
# Decision variables

X = {}
for i in range(1, I+1):
    for j in range(1, J+1):
        for k in range(1, K+1):
            x = solver.BoolVar(f'Police area {k} coordinates Police station {j}, which patrols Community area {i}')
            X[(i, j, k)] = x

Y = {}
for j in range(1, J+1):
    for j_prime in range(1, J+1):
        for k in range(1, K+1):
            y = solver.BoolVar(f'Police area {k} coordinates Police stations {j} and {j_prime}')
            Y[(j, j_prime, k)] = y
            

### Objective function
Minimizing the distance from police stations to community areas and between police stations within the same police area
<br>
min Z =  $\sum_{i=1}^{I} \sum_{j=1}^{J} \sum_{k=1}^{K}X_{i,j,k}*D_{i, j}$ + λ $\sum_{j=1}^{J} \sum_{j'=1}^{J} \sum_{k=1}^{K} Y_{j, j', k}* d_{j, j'}$ <br>

In [20]:
# Objective function

objective = solver.Objective()
objective.SetMinimization()

for i in range(1, I+1):
    for j in range(1, J+1):
        for k in range(1, K+1):
            x = X[(i, j, k)]
            D = police_stations['D_distances'][j-1][i-1]
            objective.SetCoefficient(x, D)


lbd = 0.5  #### probar valores

for j in range(1, J+1):
    for j_prime in range(1, J+1):
        if j!=j_prime:
            for k in range(1, K+1):
                y = Y[(j, j_prime, k)]
                d = police_stations['d_distances'][j-1][j_prime-1]
                objective.SetCoefficient(y, lbd*d)  

### Constraints
**_Cardinality constraints_**
<br>
[Community areas] $\sum_{j=1}^{J} \sum_{k=1}^{K} X_{i, j, k} = 1$; _i_ = 1..._I_ ‎ ‎ # Each community area is patrolled by one police station <br> 
[Police stations] $\sum_{i=1}^{I} \sum_{k=1}^{K} X_{i, j, k} \geq 1$; _j_ = 1..._J_ ‎ ‎ # Each police station patrols at least one community area <br>
[Police areas] $\sum_{i=1}^{I} \sum_{j=1}^{J} X_{i, j, k} \geq 1$; _k_ = 1..._K_ ‎ ‎ # Each police area coordinates at least one police station <br>
[Police area-Police station] $\sum_{k=1}^K X_{i, j, k} = 1; i=1..I, j = 1..J$ ‎ ‎ # A police station is only coordinated by one police area <br>

In [21]:
ca_constraints = []
for i in range(1, I+1):
    constraint = solver.Constraint(1, 1)
    for j in range(1, J+1):
        for k in range(1, K+1):
            x = X[(i, j, k)]
            constraint.SetCoefficient(x, 1)
    ca_constraints.append(constraint)
    
ps_constraints = []
for j in range(1, J+1):
    constraint = solver.Constraint(1, solver.infinity())
    for i in range(1, I+1):
        for k in range(1, K+1):
            x = X[(i, j, k)]
            constraint.SetCoefficient(x, 1)
    ps_constraints.append(constraint)
    
pa_constraints = []
for k in range(1, K+1):
    constraint = solver.Constraint(1, solver.infinity())
    for i in range(1, I+1):
        for j in range(1, J+1):
            x = X[(i, j, k)]
            constraint.SetCoefficient(x, 1)
    pa_constraints.append(constraint)


ps_pa_constraints = []
for j in range(1, J+1):
    constraint = solver.Constraint(1, 1)
    for k in range(1, K+1):
        constraint.SetCoefficient(Y[(j, j, k)], 1)
    ps_pa_constraints.append(constraint)

# ps_pa_constraints = []
# for i in range(1, I+1):
#     for j in range(1, J+1):
#         constraint = solver.Constraint(1, 1)
#         for k in range(1, K+1):
#             constraint.SetCoefficient(X[(j, j, k)], 1)
#         ps_pa_constraints.append(constraint)



# ps_pa2_constraints = []
# for j in range(1, J+1):
#     for j_prime in range(j, J+1):
#         for k in range(1, K+1):
#             constraint = solver.Constraint(1, 1)

#             constraint.SetCoefficient(Y[(j, j_prime, k)], 1)
#         ps_pa2_constraints.append(constraint)


**_Max number of X in charge of constraints_** 
<br>
[Max CA at j]  $\sum_{i=1}^{I} \sum_{k=1}^{K} X_{i, j, k} \leq 4$; _j_ = 1..._J_ ‎ ‎ # Each police station does NOT patrol more than 4 community areas <br>
[Max PS at k]  $\sum_{i=1}^{I} \sum_{j=1}^{J} X_{i, j, k} \leq 5$; _k_ = 1..._K_ ‎ ‎ # Each police area does NOT coordinate more than 5 police stations <br>

In [22]:
# TB SE PUEDE PONER EN PARÁMETROS

max1_constraints = []
for j in range(1, J+1):
    constraint = solver.Constraint(-solver.infinity(), 4)
    for i in range(1, I+1):
        for k in range(1, K+1):
            x = X[(i, j, k)]
            constraint.SetCoefficient(x, 1)
    max1_constraints.append(constraint)
    
max2_constraints = []
for j in range(1, J+1):
    constraint = solver.Constraint(-solver.infinity(), 5)
    for i in range(1, I+1):
        for k in range(1, K+1):
            x = X[(i, j, k)]
            constraint.SetCoefficient(x, 1)
    max2_constraints.append(constraint)

**_Distance constraints_**
<br>
[CA i, PS j] $\sum_{k=1}^{K} X_{i, j, k} * D_{i,j} \leq T$; _i_ = 1..._I_; _j_ = 1..._J_ ‎ ‎ # Each police station only patrols a community area if it is not too far, at _T_ seconds distance maximum <br> 
[PS j, PA k] $Y_{j,j', k} * d_{j,j'} \leq t $; _j_ = 1..._J_; _j'_ = 1..._J_, _k_ = 1.._K_ ‎ ‎ # Police area coordinates police stations that are not too far between them <br>

In [23]:
# T = 1200
# t = 1200*3

# distance1_constraints = []
# for i in range(1, I+1): 
#     for j in range(1, J+1):
#         constraint = solver.Constraint(-solver.infinity(), T)  
#         for k in range(1, K+1):
#             x = X[(i, j, k)]
#             D = police_stations['D_distances'][j-1][i-1]
#             constraint.SetCoefficient(x, D)
#         distance1_constraints.append(constraint)
        
# distance2_constraints = []                                  
# for j in range(1, J+1):
#     for j_prime in range(1, J+1):
#         for k in range(1, K+1):
#             constraint = solver.Constraint(-solver.infinity(), t)
#             y = Y[(j, j_prime, k)]
#             d = police_stations['d_distances'][j-1][j_prime-1]
#             constraint.SetCoefficient(y, d)
#             distance2_constraints.append(constraint)

**_Workload division constraints_**
<br>
[Job division at j] $ -0.015 \leq w_j - \frac{\sum_{j=1}^{J}w_j}{J} \leq 0.015$; _j_ = 1..._J_ ‎ ‎ # Balanced job division at police stations <br>
[Job division at k] $ -0.015 \leq v_k - \frac{\sum_{k=1}^{K}v_k}{K} \leq 0.015$; _k_ = 1..._K_ ‎ ‎ # Balanced job division at police areas <br>
[Job j] $ \sum_{i \in {5, 41, 47, 49, 65, 75}} \sum_{k=1}^{K} X_{i,j,k} \leq 1$; _j_ = 1..._J_ ‎ ‎ # Each police station can patrol only one community area with a high crime index <br>

Reformulation of those constraints to be included in the model

> Original <br>
$w_k - \frac 1 {23} \in [-0.015, 0.015]$ <br>
$v_k - \frac 1 {23} \in [-0.015, 0.015]$


> Equivalent <br>
$w_k \in [-0.015 + \frac 1 {23} , 0.015 + \frac 1 {23}]$ <br>
$w_k \in [-0.015 + \frac 1 5 , 0.015 + \frac 1 5]$ <br>


In [24]:

wk_constraints = []
for j in range(1, J+1):

    lower_bound = -0.015 - (1 / 23)
    upper_bound = 0.015 + (1 / 23)
    

    constraint = solver.Constraint(lower_bound, upper_bound)
    
    for i in range(1, I+1):
        for k in range(1, K+1):
            C = community_areas['criminality_index'][i-1]
            x = X[(i, j, k)]
            constraint.SetCoefficient(x, C)
    wk_constraints.append(constraint)
            

vk_constraints = []
for k in range(1, K+1):

    lower_bound = -0.015 - (1 / 5)
    upper_bound = 0.015 + (1 / 5)
    
    constraint = solver.Constraint(lower_bound, upper_bound)
    
    for i in range(1, I+1):
        for j in range(1, J+1):
            C = community_areas['criminality_index'][i-1]
            x = X[(i, j, k)]
            constraint.SetCoefficient(x, C) 
    vk_constraints.append(constraint)


workload_constraints = []
for j in range(1, J+1):
    constraint = solver.Constraint(-solver.infinity(), 1)
    for i in [5, 41, 47, 49, 65, 75]:
        for k in range(1, K+1):
            x = X[(i, j, k)]
            constraint.SetCoefficient(x, 1)
    workload_constraints.append(constraint)



**_Linking constraints:_**
<br>
[Linking] $X_{i, j, k} \leq \sum_{j'=1}^J Y_{j, j', k} $ i = 1..I; j..J; k..K;

In [25]:
# linking_constraints = []

# for i in range(1, I+1):
#     for j in range(1, J+1):
#         for k in range(1, K+1):
#             constraint = solver.Constraint(0, solver.infinity())  
#             x = X[(i, j, k)]
#             constraint.SetCoefficient(x, -1)  
#             for j_prime in range(1, J+1):
#                 constraint.SetCoefficient(Y[(j, j_prime, k)], 1) 
#             linking_constraints.append(constraint)


[R1] $Y_{j, j', k} \leq \sum_{i=1}^{I} X_{i, j, k}, \forall j, j', k$ <br>
[R2]$Y_{j, j', k} \leq \sum_{i=1}^{I} X_{i, j', k},  \forall j, j', k$ <br>
[R3]$Y_{j, j', k} \geq \sum_{i=1}^{I} X_{i, j, k} + \sum_{i=1}^{I} X_{i, j', k} - 1,\forall j, j', k$

In [26]:
# Linking constraints
# linking_constraints = []

# for j in range(1, J+1):
#     for j_prime in range(1, J+1):
#         for k in range(1, K+1):
            # # Restricción 1: Y[j, j', k] <= sum_i X[i, j, k]
            # constraint1 = solver.Constraint(0, solver.infinity())
            # constraint1.SetCoefficient(Y[(j, j_prime, k)], 1)
            # for i in range(1, I+1):
            #     constraint1.SetCoefficient(X[(i, j, k)], -1)
            # linking_constraints.append(constraint1)

            # # Restricción 2: Y[j, j', k] <= sum_i X[i, j', k]
            # constraint2 = solver.Constraint(0, solver.infinity())
            # constraint2.SetCoefficient(Y[(j, j_prime, k)], 1)
            # for i in range(1, I+1):
            #     constraint2.SetCoefficient(X[(i, j_prime, k)], -1)
            # linking_constraints.append(constraint2)

            # Restricción 3: Y[j, j', k] >= sum_i X[i, j, k] + sum_i X[i, j', k] - 1
            # constraint3 = solver.Constraint(-solver.infinity(), 1)
            # constraint3.SetCoefficient(Y[(j, j_prime, k)], -1)
            # for i in range(1, I+1):
            #     constraint3.SetCoefficient(X[(i, j, k)], 1)
            #     constraint3.SetCoefficient(X[(i, j_prime, k)], 1)
            # linking_constraints.append(constraint3)


In [27]:
# Nueva versión de restricciones para Y[j, j', k]
linking_constraints = []

for j in range(1, J+1):
    # Restricción de unicidad: Cada estación debe estar asignada a exactamente una área policial
    uniqueness_constraint = solver.Constraint(1, 1)
    for k in range(1, K+1):
        for i in range(1, I+1):
            uniqueness_constraint.SetCoefficient(X[(i, j, k)], 1)
    linking_constraints.append(uniqueness_constraint)

    for j_prime in range(1, J+1):
        for k in range(1, K+1):
            
            # Restricción 3 (corregida): Y[j, j', k] >= sum_i X[i, j, k] + sum_i X[i, j', k] - 1
            constraint3 = solver.Constraint(-solver.infinity(), 1)
            constraint3.SetCoefficient(Y[(j, j_prime, k)], -1)
            for i in range(1, I+1):
                constraint3.SetCoefficient(X[(i, j, k)], 1)
                constraint3.SetCoefficient(X[(i, j_prime, k)], 1)
            linking_constraints.append(constraint3)

            # Restricción 4: Simetría Y[j, j', k] == Y[j', j, k]
            if j < j_prime:
                constraint4 = solver.Constraint(0, 0)
                constraint4.SetCoefficient(Y[(j, j_prime, k)], 1)
                constraint4.SetCoefficient(Y[(j_prime, j, k)], -1)
                linking_constraints.append(constraint4)


SystemError: <built-in function Constraint_SetCoefficient> returned NULL without setting an exception

### Results

EXPERIMENTS: Mismo tiempo para todo, un par de minutos
En nuestro caso no hay una solución en poco tiempo
Es interesante medir en calidad (valor de la función objetivo)
Interpretar mapas cualitativamente

mismo tiempos, varios solvers

PROBAR PRIMER LOS PARÁMETROS Y LUEGO LOS SOLVERS

In [16]:
# Time limit: 10s

time_limit_ms = 10000
solver.SetTimeLimit(time_limit_ms)


result = solver.Solve()


if result == solver.ABNORMAL:
    print("Execution finished by an error")
elif result == solver.FEASIBLE:
    print("In the specified time limit the solver has found a feasible solution")
    for i in range(1, I+1):
        for j in range(1, J+1):
            for k in range(1, K+1):
                x = X[(i, j, k)]
                if x.SolutionValue() > 0:
                    print(x, x.solution_value())

    for j in range(1, J+1):
        for j_prime in range(1, J+1):
            for k in range(1, K+1):
                y = Y[(j, j_prime, k)]
                if y.SolutionValue() > 0:
                    print(y, y.solution_value()) 
    print("The value for the objective function is", objective.Value())
elif result == solver.INFEASIBLE:
    print("There is no feasible solution for the problem")
elif result == solver.NOT_SOLVED:
    print("In the specified time limit the solver has not found any feasible solution")
elif result == solver.OPTIMAL:
    print("In the specified time limit the solver has found a feasible solution")
    for i in range(1, I+1):
        for j in range(1, J+1):
            for k in range(1, K+1):
                x = X[(i, j, k)]
                if x.SolutionValue() > 0:
                    print(x, x.solution_value())

    for j in range(1, J+1):
        for j_prime in range(1, J+1):
            for k in range(1, K+1):
                y = Y[(j, j_prime, k)]
                if y.SolutionValue() > 0:
                    print(y, y.solution_value())                
    print("The optimal value for the objective function is", objective.Value())
elif result == solver.UNBOUNDED:
    print("The solution is unbounded")
else:
    print("Unknown error code")


In the specified time limit the solver has found a feasible solution
Police area 1 coordinates Police station 6, which patrols Community area 1 1.0
Police area 2 coordinates Police station 22, which patrols Community area 2 1.0
Police area 1 coordinates Police station 11, which patrols Community area 3 1.0
Police area 2 coordinates Police station 23, which patrols Community area 4 1.0
Police area 3 coordinates Police station 20, which patrols Community area 5 1.0
Police area 2 coordinates Police station 3, which patrols Community area 6 1.0
Police area 5 coordinates Police station 20, which patrols Community area 7 1.0
Police area 4 coordinates Police station 2, which patrols Community area 8 1.0
Police area 4 coordinates Police station 23, which patrols Community area 9 1.0
Police area 3 coordinates Police station 22, which patrols Community area 10 1.0
Police area 4 coordinates Police station 22, which patrols Community area 11 1.0
Police area 4 coordinates Police station 6, which pa

In [ ]:
# Time limit: 30min

time_limit_ms = 1800000
solver.SetTimeLimit(time_limit_ms)


result = solver.Solve()


if result == solver.ABNORMAL:
    print("Execution finished by an error")
elif result == solver.FEASIBLE:
    print("In the specified time limit the solver has found a feasible solution")
    for i in range(1, I+1):
        for j in range(1, J+1):
            for k in range(1, K+1):
                x = X[(i, j, k)]
                if x.SolutionValue() > 0:
                    print(x, x.solution_value())
    print("The value for the objective function is", objective.Value())
elif result == solver.INFEASIBLE:
    print("There is no feasible solution for the problem")
elif result == solver.NOT_SOLVED:
    print("In the specified time limit the solver has not found any feasible solution")
elif result == solver.OPTIMAL:
    print("In the specified time limit the solver has found a feasible solution")
    for i in range(1, I+1):
        for j in range(1, J+1):
            for k in range(1, K+1):
                x = X[(i, j, k)]
                if x.SolutionValue() > 0:
                    print(x, x.solution_value())
    print("The optimal value for the objective function is", objective.Value())
elif result == solver.UNBOUNDED:
    print("The solution is unbounded")
else:
    print("Unknown error code")


In [ ]:
# Time limit: 60min

time_limit_ms = 1800000*2
solver.SetTimeLimit(time_limit_ms)


result = solver.Solve()


if result == solver.ABNORMAL:
    print("Execution finished by an error")
elif result == solver.FEASIBLE:
    print("In the specified time limit the solver has found a feasible solution")
    for i in range(1, I+1):
        for j in range(1, J+1):
            for k in range(1, K+1):
                x = X[(i, j, k)]
                if x.SolutionValue() > 0:
                    print(x, x.solution_value())
    print("The value for the objective function is", objective.Value())
elif result == solver.INFEASIBLE:
    print("There is no feasible solution for the problem")
elif result == solver.NOT_SOLVED:
    print("In the specified time limit the solver has not found any feasible solution")
elif result == solver.OPTIMAL:
    print("In the specified time limit the solver has found a feasible solution")
    for i in range(1, I+1):
        for j in range(1, J+1):
            for k in range(1, K+1):
                x = X[(i, j, k)]
                if x.SolutionValue() > 0:
                    print(x, x.solution_value())
    print("The optimal value for the objective function is", objective.Value())
elif result == solver.UNBOUNDED:
    print("The solution is unbounded")
else:
    print("Unknown error code")


In [ ]:
# NO TIME LIMIT (con 20 min a mí ya me ha petado el ordenador)
ejecutar = False

if ejecutar:
    import time
    start_time = time.time()
    
    
    result = solver.Solve()
    if result == solver.ABNORMAL:
        print("Execution finished by an error")
    elif result == solver.FEASIBLE:
        print("In the specified time limit the solver has found a feasible solution")
        for i in range(1, I+1):
            for j in range(1, J+1):
                for k in range(1, K+1):
                    x = X[(i, j, k)]
            if x.SolutionValue()>0:
                print(x, x.solution_value())
    
        print("The value for the objective function is", objective.Value())
    elif result == solver.INFEASIBLE:
        print("There is no feasible solution for the problem")
    elif result == solver.NOT_SOLVED:
        print("In the specified time limit the solver has not found any feasible solution")
    elif result == solver.OPTIMAL:
        print("In the specified time limit the solver has found a feasible solution")
        for i in range(1, I+1):
            for j in range(1, J+1):
                for k in range(1, K+1):
                    x = X[(i, j, k)]
                    if x.SolutionValue()>0:
                        print(x, x.solution_value())
    
        print("The optimal value for the objective function is", objective.Value())
    elif result == solver.UNBOUNDED:
        print("The solution is unbounded")
    else:
        print("Unknown error code")
    
    print("--- %s seconds ---" % (time.time() - start_time))